<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Static-parameters" data-toc-modified-id="Static-parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Static parameters</a></span></li><li><span><a href="#Gate-parameters" data-toc-modified-id="Gate-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gate parameters</a></span></li><li><span><a href="#Optimization-and-fidelity" data-toc-modified-id="Optimization-and-fidelity-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Optimization and fidelity</a></span></li><li><span><a href="#Sweep-paramters" data-toc-modified-id="Sweep-paramters-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sweep paramters</a></span></li><li><span><a href="#Fidelity-vs-detuning-plot-for-paper" data-toc-modified-id="Fidelity-vs-detuning-plot-for-paper-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fidelity vs detuning plot for paper</a></span></li></ul></div>

In [ ]:
import sys
sys.dont_write_bytecode = True
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate
import qutip as qt
sys.path.append('/Users/longnguyen/Documents/GitHub/Fluxonium_berkeley/')
from Fluxonium_hamiltonians import qchard_fluxonium as fluxonium
from Fluxonium_hamiltonians import qchard_coupobj as coupobj
from Fluxonium_hamiltonians import qchard_evolgates as gates
from Fluxonium_hamiltonians import qchard_resonator as resonator
from Fluxonium_hamiltonians import qchard_rotope as op
import plotting_settings
from scipy.optimize import minimize
from qutip import*
from datetime import datetime
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']
figpath = '/Users/longnguyen/Google Drive/LBL Research/Illustration/HCF paper/'

# Static parameters

In [ ]:
# Parameters from notebook 2e. Multipath coupling. 
#Energies are in GHz

E_L1 = 0.5
E_C1 = 1.0
E_J1 = 4

E_L2 = 0.55
E_C2 = 1.0
E_J2 = 4

J_L = 0.002
J_C = 0.0115
phi_ext_A = np.pi
phi_ext_B = np.pi

method = 'propagator'

# Hilbert space.
nlev_q = 5

# Indices of the computational space.
comp_space = ['00', '01', '10', '11']
interaction = 'on'

# Instantiate objects
qubitA = fluxonium.Fluxonium(
    E_L=E_L1, E_C=E_C1, E_J=E_J1, nlev=nlev_q, phi_ext = phi_ext_A)
qubitB = fluxonium.Fluxonium(
    E_L=E_L2, E_C=E_C2, E_J=E_J2, nlev=nlev_q, phi_ext = phi_ext_B)
system = coupobj.CoupledObjects(qubitA, qubitB,
                                [qubitA, qubitB, J_L, 'flux'],
                                [qubitA, qubitB, J_C, 'charge'])

In [ ]:
print (abs((system.eigvec_nonint('00').dag() * system.eigvec('00')).data[0,0]))

# Gate parameters

In [ ]:
#Define operators
IZ = tensor(qeye(nlev_q), op.sigz(nlev_q))
ZI = tensor(op.sigz(nlev_q), qeye(nlev_q))
XI = tensor(op.sigx(nlev_q), qeye(nlev_q))
IX = tensor(qeye(nlev_q), op.sigx(nlev_q))
YI = tensor(op.sigy(nlev_q), qeye(nlev_q))
IY = tensor(qeye(nlev_q), op.sigy(nlev_q))
state_00 = system.eigvec('00')
state_10 = system.eigvec('10')
state_01 = system.eigvec('01')
state_11 = system.eigvec('11')
sup_state0 = (state_00 + state_01)/np.sqrt(2)
sup_state1 = (state_10 + state_11)/np.sqrt(2)

In [ ]:
## Here, check the drive amplitude wrt qubit A
# Calculate the drive frequency.
transition_to_drive = ('00', '10', '01')
level1, level2, level3 = transition_to_drive[0], transition_to_drive[1], transition_to_drive[2]
omega_d = abs(system.freq(level1, level3))

# Pulse shape.
shape = 'cosflattop'  # 'gauss', 'cos' for 1-cos, or 'square'
sigma = 0.25  # sigma in units of T_gate for shape=='gauss'
drag = False
drag_coeff = 0.2

T_gate = 100
T_rise = T_gate/2.0
t_points = np.linspace(0, T_gate, 2 * int(T_gate) + 1)

# Calculate the drive amplitude.
drive_amplitude = 14     # MHz
drive_amplitude = drive_amplitude*1e-3*2*np.pi


matr_el = np.abs(system.phi_ij(qubitA, level1, level2))
epsilon = drive_amplitude / abs(matr_el)

# The time-independent operator part of the drive term.
H_drive = epsilon*system.phi(0)
U_t = gates.evolution_operator_microwave_nonorm(system.H(), H_drive, t_points=t_points,T_gate=T_gate, T_rise = T_rise,shape=shape, sigma=sigma, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)

In [ ]:
# state_00 = tensor(basis(nlev_q,0),basis(nlev_q,0))
# state_10 = tensor(basis(nlev_q,1),basis(nlev_q,0))
# state_01 = tensor(basis(nlev_q,0),basis(nlev_q,1))
# state_11 = tensor(basis(nlev_q,1),basis(nlev_q,1))

sx0 = expect(IX,U_t*state_00)
sy0 = expect(IY,U_t*state_00)
sz0 = expect(IZ,U_t*state_00)
sx1 = expect(IX,U_t*state_10)
sy1 = expect(IY,U_t*state_10)
sz1 = expect(IZ,U_t*state_10)
R = 0.5*np.sqrt((sx0-sx1)**2 + (sy0-sy1)**2+(sz0-sz1)**2)

plt.plot(t_points, R)
plt.ylabel('R')
plt.xlabel('ns')
plt.ylim([0,1])

# Optimization and fidelity

In [ ]:
ZX = tensor(sigmaz(),sigmax())
U_ZX = (ZX*1j*np.pi/4.0).expm()
print (U_ZX*np.sqrt(2))

In [ ]:
omega_d = abs(system.freq(level1, level3))
T_gate = 200
T_rise = T_gate/3.0

def infidelity(x):
    drive_amplitude = x[0]  # MHz
    drive_amplitude = drive_amplitude*1e-3*2*np.pi
    epsilon = drive_amplitude / abs(matr_el)
    H_drive = epsilon*system.phi(0)
    U_t = gates.evolution_operator_microwave_nonorm(system.H(), H_drive, t_points=t_points,T_gate=T_gate, T_rise = T_rise, shape=shape, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)
    sx0 = expect(IX,U_t[-1]*state_00)
    sy0 = expect(IY,U_t[-1]*state_00)
    sz0 = expect(IZ,U_t[-1]*state_00)
    sx1 = expect(IX,U_t[-1]*state_10)
    sy1 = expect(IY,U_t[-1]*state_10)
    sz1 = expect(IZ,U_t[-1]*state_10)
    R = 0.5*np.sqrt((sx0-sx1)**2 + (sy0-sy1)**2+(sz0-sz1)**2)
    return (1.0 - R)

x0 = [13]
xopt = minimize(infidelity, x0, method ='Nelder-Mead', tol = None)
infidel = infidelity(xopt.x)
print (infidel, xopt.x)

In [ ]:
E_L1 = 0.5
E_C1 = 1.0
E_J1 = 4.0

E_L2 = 0.58
E_C2 = 1.0
E_J2 = 4.0

J_L = 0.002
J_C = 0.0115
phi_ext_A = np.pi
phi_ext_B = np.pi

method = 'propagator'

# Hilbert space.
nlev_q = 5

# Indices of the computational space.
comp_space = ['00', '01', '10', '11']
interaction = 'on'

# Instantiate objects
qubitA = fluxonium.Fluxonium(
    E_L=E_L1, E_C=E_C1, E_J=E_J1, nlev=nlev_q, phi_ext = phi_ext_A)
qubitB = fluxonium.Fluxonium(
    E_L=E_L2, E_C=E_C2, E_J=E_J2, nlev=nlev_q, phi_ext = phi_ext_B)
system = coupobj.CoupledObjects(qubitA, qubitB,
                                [qubitA, qubitB, J_L, 'flux'],
                                [qubitA, qubitB, J_C, 'charge'])

transition_to_drive = ('00', '10', '01')
level1, level2, level3 = transition_to_drive[0], transition_to_drive[1], transition_to_drive[2]
omega_d = abs(system.freq(level1, level3))

if (((system.eigvec_nonint('00').dag()*system.eigvec('00')).data[0,0])>0):
    vec00 = system.eigvec('00')
else:
    vec00 = -system.eigvec('00')
if (((system.eigvec_nonint('01').dag()*system.eigvec('01')).data[0,0])>0):
    vec01 = system.eigvec('01')
else:
    vec01 = -system.eigvec('01')
if (((system.eigvec_nonint('10').dag()*system.eigvec('10')).data[0,0])>0):
    vec10 = system.eigvec('10')
else:
    vec10 = -system.eigvec('10')
if (((system.eigvec_nonint('11').dag()*system.eigvec('11')).data[0,0])>0):
    vec11 = system.eigvec('11')
else:
    vec11 = -system.eigvec('11')
    
    
# vec00 = (system.eigvec('00'))
# vec01 = (system.eigvec('01'))
# vec10 = (system.eigvec('10'))
# vec11 = (system.eigvec('11'))

# vec00 = tensor(basis(nlev_q,0),basis(nlev_q,0))
# vec01 = tensor(basis(nlev_q,0),basis(nlev_q,1))
# vec10 = tensor(basis(nlev_q,1),basis(nlev_q,0))
# vec11 = tensor(basis(nlev_q,1),basis(nlev_q,1))

U_ideal = (vec00*vec00.dag() + vec01*vec01.dag() + vec10*vec10.dag() + vec11*vec11.dag()) \
+ 1.0j*(vec00*vec01.dag() + vec01*vec00.dag()) \
- 1.0j*(vec10*vec11.dag() + vec11*vec10.dag())

# U_ideal = (system.eigvec('00')*system.eigvec('00').dag() + system.eigvec('01')*system.eigvec('01').dag() + system.eigvec('10')*system.eigvec('10').dag() + system.eigvec('11')*system.eigvec('11').dag()) \
# + 1.0j*(system.eigvec('00')*system.eigvec('01').dag() + system.eigvec('01')*system.eigvec('00').dag()) \
# - 1.0j*(system.eigvec('10')*system.eigvec('11').dag() + system.eigvec('11')*system.eigvec('10').dag())

# print (U_ideal)
U_ideal = U_ideal / np.sqrt(2)

matr_el = system.phi_ij(qubitA, level1, level2)
omega_d = abs(system.freq(level1, level3))
T_gate = 100
T_rise = T_gate/2.0
t_points = np.linspace(0, T_gate, 2 * int(T_gate) + 1)

def infidelity(x):
    drive_amplitude = x[0]  # MHz
    drive_amplitude = drive_amplitude*1e-3*2*np.pi
    epsilon = drive_amplitude / abs(matr_el)
    H_drive = epsilon*system.phi(0) 
    U_t = gates.evolution_operator_microwave_nonorm(system.H(), H_drive, t_points=t_points,T_gate=T_gate, T_rise = T_rise, shape=shape, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)
    phase0 = np.angle(U_t[-1].matrix_element(vec00, vec00))
    phase1 = np.angle(U_t[-1].matrix_element(vec01, vec01))
    phase2 = np.angle(U_t[-1].matrix_element(vec10, vec10))
    phase3 = np.angle(U_t[-1].matrix_element(vec11, vec11))
    
    single_qu_z = 0
    single_qu_z = single_qu_z + np.exp(-1j*phase0)*vec00*vec00.dag()
    single_qu_z = single_qu_z + np.exp(-1j*phase1)*vec01*vec01.dag()
    single_qu_z = single_qu_z + np.exp(-1j*phase2)*vec10*vec10.dag()
    single_qu_z = single_qu_z + np.exp(-1j*phase3)*vec11*vec11.dag()
#     single_qu_z = single_qu_z + np.exp(1.0j*(phase0 - phase1 - phase2))*system.eigvec('11')*system.eigvec('11').dag()
    U = single_qu_z*U_t[-1] 
#     U_CNOT = U_Z2I * U * U_IX2
    P = vec00*vec00.dag() + vec01*vec01.dag() + vec10*vec10.dag() + vec11*vec11.dag()
    U_real = P * U * P
#     U_real = gates.change_operator_proj_subspace(
#         system, U=U, subspace=comp_space, interaction='on')
    op1 = U_real.dag() * U_real
    op2 = U_real * U_ideal.dag()
    fidel = (op1.tr() + (abs(op2.tr())) ** 2) / 20.0
    return abs(1.0 - fidel)

x0 = [15]
xopt = minimize(infidelity, x0, method ='Nelder-Mead', tol = None)
infidel = infidelity(xopt.x)
print (infidel, xopt.x)

In [ ]:
(1.6-0.55)/0.005
E_L_array = np.linspace(0.55,1.6,211)
# E_L_array

# Sweep paramters

In [ ]:
E_L_array = np.linspace(0.55,1.6,106)
E_L_array

In [ ]:
E_L_array = np.linspace(0.55,1.6,211)
T_gate = 200
T_rise = T_gate/3.0
t_points = np.linspace(0,T_gate, 2*int(T_gate)+1)

infidelity_array = np.zeros_like(E_L_array)
amplitude_array = np.zeros_like(E_L_array)

E_L1 = 0.5
E_C1 = 1.0
E_J1 = 4

E_C2 = 1.0
E_J2 = 4

J_L = 0.002
J_C = 0.0115
# J_C_array = np.load(figpath+'/optimal_JC.npy')
phi_ext_A = np.pi
phi_ext_B = np.pi

# Pulse shape.
shape = 'cosflattop'  # 'gauss', 'cos' for 1-cos, or 'square'
sigma = 0.25  # sigma in units of T_gate for shape=='gauss'
drag = False
drag_coeff = 0.2

# Hilbert space.
nlev_q = 5

# Indices of the computational space.
interaction = 'on'
comp_space = ['00', '01', '10', '11']

# Instantiate objects
qubitA = fluxonium.Fluxonium(
    E_L=E_L1, E_C=E_C1, E_J=E_J1, nlev=nlev_q, phi_ext = phi_ext_A)

for idx, E_L2 in enumerate(E_L_array):
    #Define qubit B
    qubitB = fluxonium.Fluxonium(
    E_L=E_L2, E_C=E_C2, E_J=E_J2, nlev=nlev_q, phi_ext = phi_ext_B)
    system = coupobj.CoupledObjects(qubitA, qubitB,
                                    [qubitA, qubitB, J_L, 'flux'],
                                    [qubitA, qubitB, J_C, 'charge'])  
    #Define drive frequency
    transition_to_drive = ('00', '10', '01')
    level1, level2, level3 = transition_to_drive[0], transition_to_drive[1], transition_to_drive[2]
    omega_d = abs(system.freq(level1, level3))
    
    matr_el = np.abs(system.phi_ij(qubitA, level1, level2))
    if (((system.eigvec_nonint('00').dag()*system.eigvec('00')).data[0,0])>0):
        vec00 = system.eigvec('00')
    else:
        vec00 = -system.eigvec('00')
    if (((system.eigvec_nonint('01').dag()*system.eigvec('01')).data[0,0])>0):
        vec01 = system.eigvec('01')
    else:
        vec01 = -system.eigvec('01')
    if (((system.eigvec_nonint('10').dag()*system.eigvec('10')).data[0,0])>0):
        vec10 = system.eigvec('10')
    else:
        vec10 = -system.eigvec('10')
    if (((system.eigvec_nonint('11').dag()*system.eigvec('11')).data[0,0])>0):
        vec11 = system.eigvec('11')
    else:
        vec11 = -system.eigvec('11')

    U_ideal = (vec00*vec00.dag() + vec01*vec01.dag() + vec10*vec10.dag() + vec11*vec11.dag()) \
    + 1.0j*(vec00*vec01.dag() + vec01*vec00.dag()) \
    - 1.0j*(vec10*vec11.dag() + vec11*vec10.dag())
    U_tar = U_ideal / np.sqrt(2)
    U_ideal = gates.change_operator_proj_subspace(
            system, U=U_tar, subspace=comp_space, interaction='on')
    
    def infidelity(x):
        drive_amplitude = x[0]  # MHz
        drive_amplitude = drive_amplitude*1e-3*2*np.pi
        epsilon = drive_amplitude / abs(matr_el)
        H_drive = epsilon*system.phi(0) 
        U_t = gates.evolution_operator_microwave_nonorm(system.H(), H_drive, t_points=t_points,T_gate=T_gate, T_rise = T_rise, shape=shape, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)
        phase0 = np.angle(U_t[-1].matrix_element(system.eigvec('00').dag(), system.eigvec('00')))
        phase1 = np.angle(U_t[-1].matrix_element(system.eigvec('01').dag(), system.eigvec('01')))
        phase2 = np.angle(U_t[-1].matrix_element(system.eigvec('10').dag(), system.eigvec('10')))
        phase3 = np.angle(U_t[-1].matrix_element(system.eigvec('11').dag(), system.eigvec('11')))

        single_qu_z = 0
        single_qu_z = single_qu_z + np.exp(-1j*phase0)*system.eigvec('00')*system.eigvec('00').dag()
        single_qu_z = single_qu_z + np.exp(-1j*phase1)*system.eigvec('01')*system.eigvec('01').dag()
        single_qu_z = single_qu_z + np.exp(-1j*phase2)*system.eigvec('10')*system.eigvec('10').dag()
        single_qu_z = single_qu_z + np.exp(-1j*phase3)*system.eigvec('11')*system.eigvec('11').dag()
#         single_qu_z = single_qu_z + np.exp(1.0j*(phase0 - phase1 - phase2))*system.eigvec('11')*system.eigvec('11').dag()
        U = single_qu_z*U_t[-1] 

        U_real = gates.change_operator_proj_subspace(
            system, U=U, subspace=comp_space, interaction='on')
        op1 = U_real.dag() * U_real
        op2 = U_real * U_ideal.dag()
        fidel = (op1.tr() + (abs(op2.tr())) ** 2) / 20.0
#         fidel = gates.fidelity_twoq_general(U_ideal, U_real)
        return abs(1.0 - fidel)
    
    if idx == 0:
        amplitude_guess = -5
    else:
        amplitude_guess = amplitude_array[idx-1]
    x0 = [amplitude_guess]
    xopt = minimize(infidelity, x0, method ='Nelder-Mead', tol = None)
    amplitude_array[idx] = xopt.x[0]
    infidelity_array[idx] = infidelity(xopt.x)
    print (idx)
    print (xopt.x)
    print(infidelity(xopt.x))
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("date and time =", dt_string)
np.save(figpath+'/CR_fidelity_vs_detuning_1GHz_200ns_full.npy', infidelity_array)

In [ ]:
plt.plot(E_L_array,infidelity_array)
plt.yscale('log')

# Fidelity vs detuning plot for paper

In [ ]:
#First, compute detuning from EL
E_L = 0.5
E_C = 1.0
E_J = 4
phi_ext = np.pi
nlev_q=5

#Qubit A
qubit = fluxonium.Fluxonium(
    E_L=E_L, E_C=E_C, E_J=E_J, nlev=nlev_q, phi_ext = phi_ext)
w1 = qubit.freq(0,1)

#Qubit B
E_L_array = np.linspace(0.55,1.6,211)
w2_array = np.zeros_like(E_L_array)
for idx, E_L in enumerate(E_L_array):
    qubit = fluxonium.Fluxonium(
    E_L=E_L, E_C=E_C, E_J=E_J, nlev=nlev_q, phi_ext = phi_ext_A)
    w2_array[idx] = qubit.freq(0,1)
detuning = w2_array-w1

In [ ]:
plt.figure(figsize = [5,4])

#100ns gate time
infidelity = np.load(figpath+'/CR_fidelity_vs_detuning_1GHz_100ns_full.npy')
plt.plot(detuning*1e3,infidelity, linewidth = 1.5, label = r'$\tau_g=100~\mathrm{ns}$')
plt.yscale('log')

#200ns gate time
infidelity = np.load(figpath+'/CR_fidelity_vs_detuning_1GHz_200ns_full.npy')
infidelity_refined = []
detuning_refined = []
for idx,error in enumerate(infidelity):
    if error < 1e-1:
        infidelity_refined = np.append(infidelity_refined,infidelity[idx])
        detuning_refined = np.append(detuning_refined,detuning[idx])
plt.plot(detuning_refined[2:]*1e3,infidelity_refined[2:], linewidth = 1.5, label = r'$\tau_g=200~\mathrm{ns}$')
plt.yscale('log')

#300ns gate time
infidelity = np.load(figpath+'/CR_fidelity_vs_detuning_1GHz_300ns_full1.npy')
infidelity_refined = []
detuning_refined = []
for idx,error in enumerate(infidelity):
    if error < 1e-1:
        if error < 2*infidelity[idx-1]:
            infidelity_refined = np.append(infidelity_refined,infidelity[idx])
            detuning_refined = np.append(detuning_refined,detuning[idx])
plt.plot(detuning_refined*1e3,infidelity_refined, linewidth = 1.5, label = r'$\tau_g=300~\mathrm{ns}$')
plt.yscale('log')

#Plotting format
plt.xlabel(r"$\Delta/2\pi~\mathrm{(MHz)}$")
plt.ylabel(r'$1-\mathcal{F}$')
plt.xlim([0,1000])
plt.ylim([5e-6,5e-1])
# plt.legend()
plt.tight_layout()
figname = 'CR_extended.pdf'
plt.savefig(figpath+figname)